<h1 style="background-color:red;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
    🟢 הסבת נקודות גבול מבוטלות לבנק"ל מודרני 🟢

In [ ]:
print('20. Canceled Border Points:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from configs import CNFG
import numpy as np
import pandas as pd
import geopandas as gpd
import dask_geopandas as daskgpd
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('  20.a Reading inputs')

<h1 style="text-align:right;font-size:100%">
נתוני שכבת נקודות היסטוריות יוצאות

In [ ]:
history_cols = ['POINT_NAME', 'ACCURACY', 'MARK_CODE', 'TALAR_NUM', 'TALAR_YEAR', 'GUSHNUM', 'GUSHSUFFIX', 'CADASTER_PROCESS', 'geometry']

history_points = daskgpd.read_file(CNFG.Local + 'Points_InProcess_History_out.gdb', layer = 'Points_InProcess_History_out', columns = history_cols, npartitions = 8).compute()

del history_cols

print('         input 1/3 read')

<h1 style="text-align:right;font-size:100%">
שכבת נקודות גבול (רצף) של תוצר

In [ ]:
BorderPoints = daskgpd.read_file(CNFG.ProductPath, layer = 'BorderPoints', columns = ['geometry'], npartitions = 8).compute()

print('         input 2/3 read')

<h1 style="text-align:right;font-size:100%">
נתוני גבולות תהליכי קדסטר של תוצר

In [ ]:
Records_cols = ['ProcessName', 'RecordGUID']

Records = pd.read_csv(CNFG.CSVPath + 'ProcessesAndRecords.csv', usecols = Records_cols)
                      
del Records_cols

print('         input 3/3 read')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('  20.b Cleaning data')

<h1 style="text-align:right;font-size:100%">
הרחבת נקודות גבול של תוצר ב-1 סנטימטר והוספת שדה קבוע לאבחנה של איחוד מרחבי בהמשך

In [ ]:
BorderPoints = BorderPoints.buffer(0.1)
BorderPoints = BorderPoints.to_frame('geometry')
BorderPoints['exist'] = 1

<h1 style="text-align:right;font-size:100%">
איחוד מרחבי בין נקודות גבול היסטוריות יוצאות לנקודות גבול ברצף קדסטרי
<h1 style="text-align:right;font-size:100%">
שמירת נקודות היסטוריות אשר לא נמצאה להן התאמה מרחבית עם נקודות ברצף.

In [ ]:
history_points = history_points.sjoin(BorderPoints, how = 'left', predicate = 'intersects')
history_points = history_points[history_points['exist'] != 1]
history_points.drop(columns = ['index_right', 'exist'], inplace = True)

del BorderPoints

<h1 style="text-align:right;font-size:100%">
הגדרת שדות של שם תהליך מבטל לכל נקודה ומספר מלא של גוש לכל נקודה

In [ ]:
history_points[['TALAR_NUM', 'TALAR_YEAR']] = history_points[['TALAR_NUM', 'TALAR_YEAR']].fillna(0)
history_points[['TALAR_NUM', 'TALAR_YEAR']] = history_points[['TALAR_NUM', 'TALAR_YEAR']].astype('Int64')
history_points['ProcessName'] = history_points['TALAR_NUM'].astype(str) + '/' + history_points['TALAR_YEAR'].astype(str)
history_points['ProcessName'] = np.where(history_points['ProcessName'] == '0/0', None, history_points['ProcessName'])

history_points[['GUSHNUM', 'GUSHSUFFIX']] = history_points[['GUSHNUM', 'GUSHSUFFIX']].fillna(0)
history_points[['GUSHNUM', 'GUSHSUFFIX']] = history_points[['GUSHNUM', 'GUSHSUFFIX']].astype('Int64')

history_points.drop(columns = ['TALAR_NUM', 'TALAR_YEAR'], inplace = True)

<h1 style="text-align:right;font-size:100%">
השלמת שדה של שם התהליך לתהליכי מפות הסדר מקרקעין ומפות קמ"ק

In [ ]:
block_regulation_mask = (history_points['ProcessName'].isna()) & (history_points['CADASTER_PROCESS'] == 1) #הסדר
history_points['ProcessName'] = np.where(block_regulation_mask,
                                         history_points['GUSHNUM'].astype(str) + '/' + history_points['GUSHSUFFIX'].astype(str) + ' הסדר',
                                         history_points['ProcessName'])



cbc_blocks_mask = (history_points['ProcessName'].isna()) & (history_points['CADASTER_PROCESS'] == 9) #קמ"ק
history_points['ProcessName'] = np.where(cbc_blocks_mask,
                                         'קמ"ק ' + history_points['GUSHNUM'].astype(str) + '/' + history_points['GUSHSUFFIX'].astype(str),
                                         history_points['ProcessName'])



del [block_regulation_mask, cbc_blocks_mask]

In [ ]:
print('  20.c Data conversion')

<h1 style="text-align:right;font-size:100%">
איחוד נקודות גבול מבוטלות עם נתוני גבולות תהליכי קדסטר

In [ ]:
history_points = history_points.merge(Records, on = 'ProcessName', how = 'left')
history_points = history_points.drop(columns = ['ProcessName', 'GUSHNUM', 'GUSHSUFFIX'])
history_points.rename(columns = {'RecordGUID' : 'RetiredByRecord'}, inplace = True)
del Records

<h1 style="text-align:right;font-size:100%">
הגדרת שדות ציון לקואורדינטות

In [ ]:
history_points['X'] = history_points.geometry.x
history_points['Y'] = history_points.geometry.y

<h1 style="text-align:right;font-size:100%">
הסרת נקודות גבול מחוץ לגבולות ישראל לפי קואורדינטות קווי רוחב

In [ ]:
southest_coord = 373520.000
northest_coord = 805000.000

history_points = history_points[history_points['Y'].between(southest_coord,  northest_coord)]

<h1 style="text-align:right;font-size:100%">
הגדרת שדה סימון הנקודה לפי מיפוי ערכים

In [ ]:
MARK_classifier = {0    : 0,
                   1    : 1,
                   2    : 2,
                   3    : 3,
                   4    : 4,
                   5    : 5,
                   6    : 6,
                   7    : 7,
                   8    : 8,
                   9    : 9,
                   10   : 10,
                   11   : 11,
                   12   : 12,
                   13   : 13,
                   14   : 14,
                   15   : 0,
                   16   : 0,
                   20   : 20,
                   50   : 0,
                   94   : 0,
                   9002 : 0,
                   9007 : 0,
                   None : 0}

history_points['MarkCode'] = history_points['MARK_CODE'].map(MARK_classifier)
history_points['MarkCode'] = history_points['MarkCode'].fillna(0)
history_points['MarkCode'] = history_points['MarkCode'].astype(int)

history_points.drop(columns = 'MARK_CODE', inplace = True)
del MARK_classifier

<h1 style="text-align:right;font-size:100%">
הגדרת שדה מקור הנקודה

In [ ]:
history_points['ACCURACY'] = history_points['ACCURACY'].fillna(0)

CAD_Sourced_cond = ~history_points['ACCURACY'].isin([0, None, np.nan])
digitaized_cond = history_points['CADASTER_PROCESS'] == 10

history_points['DataSource'] = np.where(CAD_Sourced_cond, 5, 4)
history_points['DataSource'] = np.where(digitaized_cond, 3, 4)
history_points['DataSource'] = history_points['DataSource'].fillna(4)
history_points['DataSource'] = history_points['DataSource'].astype(int)

history_points.drop(columns = 'CADASTER_PROCESS', inplace = True)
del CAD_Sourced_cond

<h1 style="text-align:right;font-size:100%">
שינוי שמות של שדות

In [ ]:
history_points.rename(columns = {'POINT_NAME' : 'Name'}, inplace = True)

<h1 style="text-align:right;font-size:100%">
הגדרת שדה לסיווג הנקודה

In [ ]:
ACCURACY_classifier = {0    : 24,
                       1    : 1,
                       2    : 13,
                       3    : 13,
                       4    : 13,
                       5    : 13,
                       6    : 24,
                       7    : 24,
                       8    : 24,
                       9    : 24,
                       12   : 12,
                       13   : 13,
                       24   : 24,
                       None : 24}

history_points['Class'] = history_points['ACCURACY'].map(ACCURACY_classifier)
history_points['Class'] = history_points['Class'].fillna(24)
history_points['Class'] = history_points['Class'].astype(int)

history_points.drop(columns = 'ACCURACY', inplace = True)
del ACCURACY_classifier

<h1 style="text-align:right;font-size:100%">
השלמת שדות של תהליך מבטל לנקודה על ידי תהליך מבטל פיקטיבי

In [ ]:
history_points['RetiredByRecord'] = history_points['RetiredByRecord'].fillna(CNFG.FabricatedCancelProcessGUID)

<h1 style="text-align:right;font-size:100%">
השלמת שדה של תהליך יוצר לנקודה על ידי תהליך יוצר פיקטיבי

In [ ]:
history_points['CreatedByRecord'] = CNFG.FabricatedCreateProcessGUID

In [ ]:
history_points['IsFixed'] = 0

<h1 style="text-align:right;font-size:100%">
הגדרת שדות מספריים לשדות מסוג מספר שלם

In [ ]:
int_cols = ['MarkCode', 'DataSource', 'Class', 'IsFixed']
history_points[int_cols] = history_points[int_cols].astype('Int64')

del int_cols

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾

In [ ]:
print('  20.e Exporting results')

In [ ]:
history_points.to_file(CNFG.Results, layer = 'CancelBorderPoints', driver = 'OpenFileGDB', engine = 'pyogrio', mode = 'w')

In [ ]:
print('  20.f Appending results to product')

In [ ]:
subprocess.call([CNFG.BatchPath + "CancelBorderPoints-Save as Feature Class.bat"]);